### Import Modules

In [133]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [138]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [139]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [140]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if False: #p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  2.11800003052
Compiling results...
Done.

Total time (s):  9.625 



In [77]:
def get_delta_su2_correction(self):
    lam2_chi = self.get_phys_point_data('lam2_chi')
    eps2_pi = (self.get_phys_point_data('mpi'))**2 / lam2_chi
    eps2_k = (self.get_phys_point_data('mk'))**2 / lam2_chi
    fkfpi = self.extrapolate_to_phys_point()
    
    R = gv.gvar('35.7(2.6)') # From FLAG
    eps_su2 = np.sqrt(3)/(4.0 *R)
    
    delta = np.sqrt(3) *eps_su2 *(
        - (4.0/3.0) *(fkfpi - 1)
        + (4.0/3.0) *(eps2_k - eps2_pi - eps2_pi *np.log(eps2_k/eps2_pi))
    )
    return delta

get_delta_su2_correction(bootstrapper)

1.1868(33)

In [70]:
np.sqrt(1 + get_delta_su2_correction(bootstrapper))

0.99895(11)

### Or make all fits

In [123]:
p_dict = {
    'order' : {
        'vol' : 0 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [124]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  1.85300016403
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.1949999332 

xpt nnlo
Using default prior.
Making fits...
100% complete Time (s):  3.74699997902
Compiling results...
Done.
Saving...
Done.

Total time (s):  14.6010000706 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  1.72300004959
Compiling results...
Done.
Saving...
Done.

Total time (s):  9.43799996376 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  3.70899987221
Compiling results...
Done.
Saving...
Done.

Total time (s):  11.4040000439 

ma nlo
Using default prior.
Making fits...
100% complete Time (s):  2.40499997139
Compiling results...
Done.
Saving...
Done.

Total time (s):  55.0609998703 

ma nnlo
Using default prior.
Making fits...
100% complete Time (s):  4.86800003052
Compiling results...
Done.
Saving...
Done.

Total time (s):  56.9520001411 

ma-taylor nlo
Using defau

In [38]:
#bootstrapper.fits[0].show_plots(view='std')

In [109]:
bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=True)

-0.00240(27)

In [112]:
output = 3
output *= 2
output

6